# AQE

**Turned off AQE**

In [0]:
spark.conf.set('spark.sql.adaptive.enabled', False)

In [0]:
from pyspark.sql.functions import *

In [0]:
df = (spark.read.format('csv')
                .option('inferSchema',True)
                .option('header', True)
                .load('/FileStore/tables/BigMart_Sales.csv'))

In [0]:
df.rdd.getNumPartitions()

Out[5]: 1

In [0]:
df_new = df.groupBy('Item_Fat_Content').count()

In [0]:
df_new.display()

Item_Fat_Content,count
low fat,112
Low Fat,5089
LF,316
Regular,2889
reg,117


%md
**Turned on AQE**

In [0]:
spark.conf.set('spark.sql.adaptive.enabled', True)

In [0]:
df_new = df.groupBy('Item_Fat_Content').count()

In [0]:
df_new.display()

Item_Fat_Content,count
low fat,112
Low Fat,5089
LF,316
Regular,2889
reg,117


Now you will see a step called AQEShuffleRead in which, after the usual exchange step that performs 200 partitions for wide transformations, it reduces those into one.

AQE will help on:

1. Coalescing partitions
2. Skewed data
3. Join Optimization

## Join Optimizations

In [0]:
# Big DataFrame
df_transactions = spark.createDataFrame([
    (1, "US", 100),
    (2, "IN", 200),
    (3, "UK", 150),
    (4, "US", 80),
], ["id", "country_code", "amount"])

# Small DataFrame
df_countries = spark.createDataFrame([
    ("US", "United States"),
    ("IN", "India"),
    ("UK", "United Kingdom"),
], ["country_code", "country_name"])

In [0]:
df_join = (df_transactions.join(
                      df_countries
                      , df_transactions['country_code'] == df_countries['country_code']
                      , how='inner' 
          )
)

In [0]:
df_join.display()

id,country_code,amount,country_code,country_name
2,IN,200,IN,India
3,UK,150,UK,United Kingdom
1,US,100,US,United States
4,US,80,US,United States
